In [1]:
from elasticsearch import Elasticsearch
es = Elasticsearch([{'host': 'localhost', 'port': 9200}])

In [2]:
# es.index(index='posts', doc_type='blog', id=2, body={
#     'author': 'Benjamin Pollack',
#     'blog': 'bitquabit',
#     'title': 'Having Fun: Python and Elasticsearch',
#     'topics': ['elasticsearch', 'python', 'parseltongue'],
#     'awesomeness': 0.7
# })

In [3]:
from sklearn.datasets import fetch_20newsgroups
twentyng_data = fetch_20newsgroups(subset='test',remove=('headers','footers','quotes'))

In [4]:
data = twentyng_data.data

In [5]:
# es.index(index='index_20ng', doc_type='20_ng', body={
#     'doc_id': 1,
#     'doc_text': data[0]
# })

In [6]:
# es.index(index='index_20ng', doc_type='20_ng', body={
#     'doc_id': 2,
#     'doc_text': data[1]
# })

In [7]:
# es.get(index='index_20ng', doc_type='20_ng',id='LKL2eGIBw2ZjxL_NkJl2')

In [8]:
#es.search(index='index_20ng',q='doc_id:"1"')

In [9]:
### Pushing twenty ng data to its index
for index in range(len(data)):
    es.index(index='index_20ng', doc_type='20_ng',id=index,body={
        'doc_id': index+1,
        'doc_text': data[index]
    })
    


In [55]:
import os
def get_filenames(folder):

    
    filenames= os.listdir(folder)
    return filenames

In [56]:
## Pushing DUC data to its index
path = "Downloads/DUC2001/"
filenames = get_filenames(path)

In [112]:
from xml.etree import cElementTree as ET


def extract_parse_xml(xml,filename):
    print(filename)
    dict_res = {}
    #parser = etree.XMLParser(recover=True)
    #doc = etree.fromstring(xml, parser=parser)
    #doc = ET.fromstring(xml)
    #dict_res['doc_id'] = doc.find('DOCNO').text
    dict_res['doc_id'] = filename
    #print("doc_id:",dict_res['doc_id'])
    #dict_res['text'] = doc.find('TEXT').text
    dict_res['text'] = xml
    #print("doc_text:",dict_res['text'])
    ##extract summaries too
    summary_filename = filename.lower()+".txt" 
    if not os.path.exists(path +'/Summaries/'+summary_filename):
        dict_res['gold_summary'] = ''
    else:
        fd = open(path +'/Summaries/'+summary_filename, "r+")
        fileCon = fd.read()
        #print(fileCon)
        dict_res['gold_summary'] = fileCon
    #print("gold_summary:",dict_res['gold_summary'])
    return dict_res
        

In [113]:
fileCon = ''
i = 0
for filename in filenames:
    if filename=='.DS_Store' or filename=='annotations.txt':
        continue
    #filename = 'FBIS-41815'
    fd = open(path +'/'+ filename, "r+")
           
    fileCon = fd.read()
    fd.close()
    #print("filename:",filename)
    #print(fileCon)
    result_dict = extract_parse_xml(fileCon,filename)
    #print("result_dict",result_dict)
    es.index(index='index_duc', doc_type='duc', id=result_dict["doc_id"], body={
        'doc_id': result_dict["doc_id"],
        'doc_text': result_dict["text"],
        'gold_summary': result_dict["gold_summary"]
    })
    #break
    

AP830325-0143
AP880217-0175
AP880318-0051
AP880330-0119
AP880331-0140
AP880409-0015
AP880419-0131
AP880510-0178
AP880517-0226
AP880520-0264
AP880601-0040
AP880613-0161
AP880623-0135
AP880629-0159
AP880630-0295
AP880705-0006
AP880705-0018
AP880705-0109
AP880714-0142
AP880801-0195
AP880811-0299
AP880816-0234
AP880901-0052
AP880902-0062
AP880903-0092
AP880913-0129
AP880913-0204
AP880914-0027
AP880914-0079
AP880926-0203
AP880927-0089
AP880927-0117
AP880928-0054
AP880928-0146
AP881009-0072
AP881017-0235
AP881018-0136
AP881126-0007
AP881206-0114
AP881210-0115
AP881211-0027
AP881216-0017
AP881222-0089
AP881222-0119
AP881222-0126
AP881227-0185
AP890111-0217
AP890111-0227
AP890117-0132
AP890131-0280
AP890227-0016
AP890228-0019
AP890302-0063
AP890307-0150
AP890313-0198
AP890314-0237
AP890316-0018
AP890322-0010
AP890325-0029
AP890326-0081
AP890403-0123
AP890404-0260
AP890501-0176
AP890502-0205
AP890511-0126
AP890529-0030
AP890704-0043
AP890708-0135
AP890714-0129
AP890719-0225
AP890722-0081
AP8908

IsADirectoryError: [Errno 21] Is a directory: 'Downloads/DUC2001/kartest'

In [72]:
if not os.path.exists('Downloads/DUC2001/Summaries/ap830325-0143.txt'):
    print(True)

True


In [114]:
es.search(index='index_duc')

{'_shards': {'failed': 0, 'skipped': 0, 'successful': 5, 'total': 5},
 'hits': {'hits': [{'_id': 'AP880330-0119',
    '_index': 'index_duc',
    '_score': 1.0,
    '_source': {'doc_id': 'AP880330-0119',
     'doc_text': "\n<DOC>\n<DOCNO> AP880330-0119 </DOCNO>\n<FILEID>AP-NR-03-30-88 1245EST</FILEID>\n<FIRST>r w AM-CensusAdjustment     03-30 0390</FIRST>\n<SECOND>AM-Census Adjustment,390</SECOND>\n<HEAD>Adjusting the Census: Little Difference</HEAD>\n<BYLINE>By RANDOLPH E. SCHMID</BYLINE>\n<BYLINE>Associated Press Writer</BYLINE>\n<DATELINE>WASHINGTON (AP) </DATELINE>\n<TEXT>\n   If the two sides trying to force changes in\nthe 1990 census both get their way, the results would nearly\nbalance one another, a population expert said Wednesday.\n   The Census Bureau is under pressure to exclude illegal aliens\nfrom its national head count. Traditionally, it counts everyone\nliving in the country.\n   Groups which have filed suit to ignore the aliens contend large\nconcentrations of them co

In [111]:
es.delete(index='index_duc',doc_type='duc')

TypeError: delete() missing 1 required positional argument: 'id'